In [15]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from uncertainties import ufloat
from uncertainties.umath import *  # sin(), etc.
import uncertainties.unumpy as un

df_placa = pd.read_excel('D:/UNESP/EngMec-UNESP/Lab. Mec Flu II/Lab 1 - Medidores de vazão/dados.xlsx')
df_vent = pd.read_excel('D:/UNESP/EngMec-UNESP/Lab. Mec Flu II/Lab 1 - Medidores de vazão/dados.xlsx', sheet_name='venturi')

In [16]:
df_placa

,h1,h2,del,ve
0,18.8,20.7,1.9,3.40
1,16.5,22.9,6.4,5.94
2,14.2,25.2,11.0,7.65
3,12.5,26.8,14.3,8.60
4,10.0,29.8,19.8,9.93


In [17]:
print(df_placa.to_latex(index=False))

\begin{tabular}{rrrr}
\toprule
  h1 &   h2 &  del &   ve \\
\midrule
18.8 & 20.7 &  1.9 & 3.40 \\
16.5 & 22.9 &  6.4 & 5.94 \\
14.2 & 25.2 & 11.0 & 7.65 \\
12.5 & 26.8 & 14.3 & 8.60 \\
10.0 & 29.8 & 19.8 & 9.93 \\
\bottomrule
\end{tabular}



In [18]:
df_vent

,h1,h2,del,ve
0,19.3,20.5,1.2,3.60
1,17.8,22.0,4.2,6.40
2,15.9,24.0,8.1,8.80
3,14.1,26.0,11.9,10.58
4,9.5,30.8,21.3,13.00


In [19]:
print(df_vent.to_latex(index=False))

\begin{tabular}{rrrr}
\toprule
  h1 &   h2 &  del &    ve \\
\midrule
19.3 & 20.5 &  1.2 &  3.60 \\
17.8 & 22.0 &  4.2 &  6.40 \\
15.9 & 24.0 &  8.1 &  8.80 \\
14.1 & 26.0 & 11.9 & 10.58 \\
 9.5 & 30.8 & 21.3 & 13.00 \\
\bottomrule
\end{tabular}



In [20]:
rho_ar = 1.20
rho_agua = 997
mi = 1.80e-5
g = 9.81

def calc_vazão(Dt, D1, delta_h):
    
    beta = Dt/D1
    At = np.pi*(Dt**2)/4
    A1 = np.pi*(D1**2)/4

    delta_P = rho_agua*g*(delta_h/100)

    lista_m = []
    for j in range(len(delta_P)):
        m_dot  = 10.0
        for i in range(100):

            v = m_dot/(rho_ar*A1)
            Re = (rho_ar*v*D1)/mi
            C = 0.5959 + (0.0312*(beta**2.1)) - (0.184*beta**8) + ((91.71*beta**2.5)/(Re**0.75))
            K = C/np.sqrt(1 - beta**4)
            m_dot_new = K*At*np.sqrt(2*rho_ar*(delta_P[j]))
            
            if abs(m_dot - m_dot_new) < 1e-8:
                break
            else:
                m_dot = m_dot_new

        lista_m.append(m_dot_new)
        
    lista_m = np.array(lista_m)
    lista_Q = lista_m/rho_ar
    lista_V = lista_Q/A1

    return lista_m, lista_Q, lista_V #kg/s, m^3/s, m/s

In [21]:
#Placa de orificio
placa = calc_vazão(0.0257, 0.0385, df_placa['del'].values)
placa

(array([0.00768234, 0.01389186, 0.01812435, 0.02062295, 0.0242119 ]),
 array([0.00640195, 0.01157655, 0.01510362, 0.0171858 , 0.02017659]),
 array([ 5.49921472,  9.94415423, 12.97387612, 14.76244508, 17.33150759]))

In [22]:
res_p = pd.DataFrame({'teo':placa[2], 'exp': df_placa['ve'].values, 
                     'erro':abs(df_placa['ve'].values - placa[2])/placa[2]*100})
res_p

,teo,exp,erro
0,5.499215,3.40,38.172991
1,9.944154,5.94,40.266413
2,12.973876,7.65,41.035355
3,14.762445,8.60,41.744068
4,17.331508,9.93,42.705504


In [23]:
print(res_p.to_latex(index=False))

\begin{tabular}{rrr}
\toprule
      teo &  exp &      erro \\
\midrule
 5.499215 & 3.40 & 38.172991 \\
 9.944154 & 5.94 & 40.266413 \\
12.973876 & 7.65 & 41.035355 \\
14.762445 & 8.60 & 41.744068 \\
17.331508 & 9.93 & 42.705504 \\
\bottomrule
\end{tabular}



In [24]:
#Tubo de venturi
tubo = calc_vazão(0.0244, 0.0385, df_vent['del'].values)
tubo

(array([0.00542018, 0.00996726, 0.01375287, 0.01661733, 0.02214316]),
 array([0.00451682, 0.00830605, 0.01146073, 0.01384778, 0.01845263]),
 array([ 3.87990403,  7.1348258 ,  9.84466122, 11.89511582, 15.85064489]))

In [25]:
res_t = pd.DataFrame({'teo':tubo[2], 'exp': df_vent['ve'].values, 
                     'erro':abs(df_vent['ve'].values - tubo[2])/tubo[2]*100})
res_t

,teo,exp,erro
0,3.879904,3.60,7.214200
1,7.134826,6.40,10.299141
2,9.844661,8.80,10.611449
3,11.895116,10.58,11.055931
4,15.850645,13.00,17.984410


In [26]:
print(res_t.to_latex(index=False))

\begin{tabular}{rrr}
\toprule
      teo &   exp &      erro \\
\midrule
 3.879904 &  3.60 &  7.214200 \\
 7.134826 &  6.40 & 10.299141 \\
 9.844661 &  8.80 & 10.611449 \\
11.895116 & 10.58 & 11.055931 \\
15.850645 & 13.00 & 17.984410 \\
\bottomrule
\end{tabular}



In [27]:
#ERROS ENTRE AS PLACAS
res_t = pd.DataFrame({'placa':placa[2], 'tubo': tubo[2], 
                     'erro':abs(tubo[2] - placa[2])/placa[2]*100})
res_t

,placa,tubo,erro
0,5.499215,3.879904,29.446217
1,9.944154,7.134826,28.251054
2,12.973876,9.844661,24.119352
3,14.762445,11.895116,19.423132
4,17.331508,15.850645,8.544339


In [28]:
print(res_t.to_latex(index=False))

\begin{tabular}{rrr}
\toprule
    placa &      tubo &      erro \\
\midrule
 5.499215 &  3.879904 & 29.446217 \\
 9.944154 &  7.134826 & 28.251054 \\
12.973876 &  9.844661 & 24.119352 \\
14.762445 & 11.895116 & 19.423132 \\
17.331508 & 15.850645 &  8.544339 \\
\bottomrule
\end{tabular}

